# Импорт библиотек и чтение файлов

In [32]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np
import string
from tqdm import tqdm_notebook as tqdm
import re

#NLTK
import nltk
from nltk.corpus import stopwords
nltk.download("english")
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index


In [2]:
movies_df = pd.read_csv(r'movies_metadata.csv')
rating_df = pd.read_csv(r'ratings.csv')

C:\Users\isaev\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Preprocessing

*Объединим два датафрейма в один*

In [3]:
df = movies_df.join(rating_df)
df.shape

(45466, 28)

*Проверим на наличие NaN

In [4]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
userId                       0
movieId                      0
rating                       0
timestamp                    0
dtype: int64

In [5]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,userId,movieId,rating,timestamp
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1,110,1.0,1425941529
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1,147,4.5,1425942435


In [6]:
df['original_title'] = df['title']

*Удалим столбцы с большим количеством NaN и столбцы, которые нам не особо нужны, чтоб уменьшим размер датафрейма*

In [7]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'userId', 'movieId', 'rating',
       'timestamp'],
      dtype='object')

In [8]:
df = df.drop(['belongs_to_collection', 'homepage', 'budget', 'imdb_id', 'poster_path', 'release_date',
             'revenue', 'status', 'homepage', 'imdb_id', 'poster_path', 'popularity', 'revenue', 'title', 'video',], axis=1)

In [9]:
df = df.fillna(0)
df.isnull().sum()

adult                   0
genres                  0
id                      0
original_language       0
original_title          0
overview                0
production_companies    0
production_countries    0
runtime                 0
spoken_languages        0
tagline                 0
vote_average            0
vote_count              0
userId                  0
movieId                 0
rating                  0
timestamp               0
dtype: int64

*Функця препроцессинга*

In [10]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    """
    На вход: датафрейм с описанием и названием фильмов,
    также пробрасываются стоп-слова и класс для лемматизации текстов
    
    На выходе: список с предобработанными текстовыми данными
    """
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]
        # Lowercase & Drop stopwords and punctuation
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        # stem_text = " ".join([stemmer.stem(word) for word in stop_free.split(' ')])
        
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    return new_text

In [11]:
text_overview = preprocessing(df["overview"])
text_title = preprocessing(df["original_title"])

df["overview"] = text_overview
df["original_title"] = text_title 

# Закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_title + text_overview

C:\Users\isaev\AppData\Local\Temp/ipykernel_6248/3915145218.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(texts))):


  0%|          | 0/45466 [00:00<?, ?it/s]

  0%|          | 0/45466 [00:00<?, ?it/s]

In [12]:
text_prepr = [i.split() for i in text_prepr]

In [13]:
len(text_prepr)

90932

# Word2Vec

In [21]:
model = Word2Vec(text_prepr, 
                 window=10, 
                 min_count=2)

In [22]:
# Построим словарь из последовательности предложений

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 40606


*Обучим модель W2V для поиска наиболее похожих слов*

In [18]:
model.train(text_prepr, total_examples=len(text_prepr), epochs=5)

(7225193, 7531220)

In [23]:
model.wv.most_similar("snow")

[('anni', 0.8940200209617615),
 ('attacked', 0.8902016878128052),
 ('detention', 0.8898559212684631),
 ('sabbath', 0.8832044005393982),
 ('homoerotic', 0.8816789388656616),
 ('nameless', 0.8790510892868042),
 ('im', 0.8767324090003967),
 ('pascal', 0.8766116499900818),
 ('suitor', 0.8761287331581116),
 ('loose', 0.8735535144805908)]

In [24]:
model.wv.most_similar("power")

[('adjust', 0.9382202625274658),
 ('gregoire', 0.9204652905464172),
 ('revers', 0.9076642990112305),
 ('th', 0.899091899394989),
 ('echo', 0.8975561261177063),
 ('avaricious', 0.8943937420845032),
 ('bronx', 0.8850521445274353),
 ('angelov', 0.8840415477752686),
 ('lise', 0.8826701045036316),
 ('toth', 0.8815551400184631)]

In [26]:
model.wv.most_similar("woman")

[('iv', 0.8626662492752075),
 ('buzzy', 0.8585712313652039),
 ('chase', 0.8202930092811584),
 ('kronos', 0.8173089027404785),
 ('selena', 0.8148725628852844),
 ('goodnight', 0.8120730519294739),
 ('oz', 0.8115226626396179),
 ('shalu', 0.8091319799423218),
 ('foote', 0.8062674403190613),
 ('gulliver', 0.8041901588439941)]

## Tockenizer & embeding

In [27]:
tokenizer = Tokenizer()

In [28]:
# Обновляем внутренний словарь на основе списка текстов.
tokenizer.fit_on_texts(text_prepr)
vocab_size = len(tokenizer.word_index) + 1

print("Total words:", vocab_size)

Total words: 71878


*Создание матрицы эмбедингов*

*Преобразуем каждый текст в текстах в последовательность целых чисел. 
texts_to_sequences берет каждое слово в тексте и заменяет его соответствующим целочисленным значением из словаря word_index.*

*pad_sequences используется для обеспечения того, чтобы все последовательности в списке имели одинаковую длину.*



In [39]:
data = pad_sequences(tokenizer.texts_to_sequences(text_prepr), maxlen=100)

In [43]:
data.shape

(90932, 100)

*Функция построения матрицы эмбедингов*

In [45]:
def embedding_matrix(tock, model):
    
    """
    На вход: словарь tokenizer.word_index 
    модель word2vec
    
    На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((vocab_size, 100))

    for word, i in tock.items():
        if word in model:
            embedding_matrix[i] = model[word]
    return embedding_matrix

In [46]:
embedding_matrix = embedding_matrix(tokenizer.word_index, model.wv)

In [47]:
embedding_matrix.shape

(71878, 100)